# modelo 07 - yolooo

yolo > mask detection > count

- https://chatgpt.com/share/67dc37e8-77b0-8009-93fa-b2e6ce5ee496

In [22]:
import pandas as pd

In [23]:
df_train = pd.read_pickle('Model_Train.pkl')
df_val = pd.read_pickle('Model_Val.pkl')

In [24]:
set(df_train["img_placement"]) | set(df_val["img_placement"])

{'S-unknown', 'openspace', 'r_openspace', 'roof'}

In [25]:
set(df_train["img_origin"]) | set(df_val["img_origin"])

{'D', 'S'}

In [26]:
def hot_encod_img_placement(df):
    extra_features_all = []
    for i, row in df.iterrows():
        extra_features = []
        if row["img_placement"] == "roof":
            extra_features += [1, 0, 0, 0]
        elif row["img_placement"] == "r_openspace":
            extra_features += [0, 1, 0, 0]
        elif row["img_placement"] == "openspace":
            extra_features += [0, 0, 1, 0]
        elif row["img_placement"] == "S-unknown":
            extra_features += [0, 0, 0, 1]
        else:
            extra_features += [0, 0, 0, 0]
        if row["img_origin"] == "D":
            extra_features += [1, 0]
        elif row["img_origin"] == "S":
            extra_features += [0, 1]
        else:
            extra_features += [0, 0] 
        extra_features_all.append(extra_features)
    
    df["more_features"] = extra_features_all

    df.drop(columns=["img_origin"], inplace=True)
    df.drop(columns=["img_placement"], inplace=True)

hot_encod_img_placement(df_train)
hot_encod_img_placement(df_val)

In [27]:
df_train.sample(3)

,img_id,nr_boil,polygons_boil,nr_pan,polygons_pan,more_features
294,IDsbMkn,[],[],[3],"[[(265.216, 193.87733333333333), (288.0, 211.6...","[1, 0, 0, 0, 1, 0]"
66,IDxXu4f,[],[],"[1, 3]","[[(302.08, 363.6906666666667), (334.464, 328.1...","[0, 1, 0, 0, 1, 0]"
963,IDA2DWMtxM9D,[],[],"[1, 2]","[[(107.392, 287.9146666666667), (131.712, 273....","[1, 0, 0, 0, 1, 0]"


In [28]:
def filter(df):
    rows_to_keep = []

    for i, row in df.iterrows():
        keep = True
        
        nr_boils = row["nr_boil"]
        nr_pans = row["nr_pan"]

        for ii in nr_boils:
            if ii > 1:
                keep = False
                break
        for ii in nr_pans:
            if ii > 1:
                keep = False
                break

        if keep:
            rows_to_keep.append(i)
        
    return df.loc[rows_to_keep]

df_train = filter(df_train)
df_val = filter(df_val)

In [29]:
df_train.sample(3)

,img_id,nr_boil,polygons_boil,nr_pan,polygons_pan,more_features
774,IDYcSSKa8TL3,[],[],[1],"[[(200.904, 161.224), (225.86399999999998, 180...","[1, 0, 0, 0, 1, 0]"
2215,IDdVlhAjGxoIoje,[1],"[[(84.42965204236006, 319.87620889748547), (11...",[],[],"[0, 0, 0, 1, 0, 1]"
606,IDko1t1FcaJu0,[],[],[1],"[[(347.1515151515151, 251.14661274014156), (36...","[1, 0, 0, 0, 1, 0]"


---

## get ready for yolo

In [30]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

folder = "val"
path = f"ModelBuild/model09/{folder}"

"""def polygons_to_segmentation_mask(name, polygons):
    # 512 x 512
    image = np.zeros((512, 512), dtype=np.uint8)
    for polygon in polygons[0]:
        poly = np.array([polygon], dtype=np.int32)
        cv2.fillPoly(image, [poly], 1)
    for polygon in polygons[1]:
        poly = np.array([polygon], dtype=np.int32)
        cv2.fillPoly(image, [poly], 2)

    # save image as name_mask.png
    cv2.imwrite(f"{path}/masks/{name}.png", image)

def resize_image(name, size=(512, 512)):
    image_file = f"images/{name}.jpg"
    image = cv2.imread(image_file)
    resized_image = cv2.resize(image, size)
    cv2.imwrite(f"{path}/images/{name}.png", resized_image)

for _, row in df_val.iterrows():

    if _ % 100 == 0:
        print(_, len(df_val))

    polygons_to_segmentation_mask(row["img_id"], [row["polygons_boil"], row["polygons_pan"]])
    resize_image(row["img_id"])

from ultralytics.data.converter import convert_segment_masks_to_yolo_seg

convert_segment_masks_to_yolo_seg(masks_dir=f"/Users/hugover/Desktop/CAA01/{path}/masks",
                                  output_dir=f"/Users/hugover/Desktop/CAA01/{path}/labels", classes=2)"""
''

''

In [31]:
from ultralytics import YOLO


# Load YOLOv8 Model
"""model = YOLO("yolo11m-seg")

# Train the model
model.train(
    data='ModelBuild/model09/data.yaml',  # Path to your data.yaml
    epochs=50,                           # Number of epochs
    imgsz=512,                           # Image size
    batch=32,                            # Batch size
    augment=True,                        # Enable data augmentation
    project='ModelBuild',                # Path where training results will be saved (logs, checkpoints)
    name='model09_yolo',                 # Name of the specific experiment folder (creates a new folder)
    exist_ok=True,                         # Overwrite the folder if it already exists
    patience=7,                          # Stop training if loss does not improve for this number of consecutive epochs
)"""
''

''

In [32]:
from ultralytics import YOLO

# Load the best trained model
model = YOLO("ModelBuild/model09_yolo/weights/best.pt")

# Run validation
metrics = model.val()

Ultralytics 8.3.94 🚀 Python-3.10.16 torch-2.6.0 CPU (Apple M1)
YOLO11m-seg summary (fused): 138 layers, 22,336,854 parameters, 0 gradients, 123.0 GFLOPs


val: Scanning /Users/hugover/Desktop/CAA01/ModelBuild/model09/val/labels.cache... 383 images, 0 backgrounds, 0 corrupt: 100%|██████████| 383/383 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/24 [00:00<?, ?it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   4%|▍         | 1/24 [00:14<05:31, 14.41s/it]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   8%|▊         | 2/24 [00:28<05:16, 14.40s/it]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [05:56<00:00, 14.87s/it]


                   all        383        552      0.772      0.586      0.664      0.374      0.773      0.587      0.671      0.328
                  boil         63         84      0.696      0.429      0.505      0.198       0.71       0.44      0.537      0.169
                   pan        323        468      0.849      0.744      0.822       0.55      0.837      0.734      0.806      0.488
Speed: 0.7ms preprocess, 914.1ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/segment/val


---
---

submissao

In [34]:
test = pd.read_csv("Test.csv")
submission = pd.read_csv("zindi_files/SampleSubmission.csv")

def resize_image(name, size=(512, 512)):
    image_file = f"images/{name}.jpg"
    image = cv2.imread(image_file)
    resized_image = cv2.resize(image, size)
    cv2.imwrite(f"ModelBuild/model09_subm/{name}.png", resized_image)

for i, row in test.iterrows():
    if i % 100 == 0:
        print(i, len(test))

    resize_image(row["ID"])
    results = model(f"ModelBuild/model09_subm/{row['ID']}.png")
    result = results[0]

    # Count occurrences of each detected class
    class_counts = {}

    for cls in result.boxes.cls.tolist():  # Get list of detected class indices
        class_counts[int(cls)] = class_counts.get(int(cls), 0) + 1

    # Map class indices to class names
    class_names = model.names  # Dictionary mapping class index to name
    named_counts = {class_names[k]: v for k, v in class_counts.items()}

    if "boil" in named_counts:
        submission.loc[submission["ID"] == f'{row["ID"]}_boil', "Target"] = named_counts["boil"]
    
    if "pan" in named_counts:
        submission.loc[submission["ID"] == f'{row["ID"]}_pan', "Target"] = named_counts["pan"]

submission.head(10)

0 1107

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model09_subm/ID00qprY.png: 512x512 1 pan, 336.4ms
Speed: 0.8ms preprocess, 336.4ms inference, 5.8ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model09_subm/ID01AciUc.png: 512x512 2 pans, 338.2ms
Speed: 0.9ms preprocess, 338.2ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model09_subm/ID0328D.png: 512x512 4 pans, 309.5ms
Speed: 0.9ms preprocess, 309.5ms inference, 1.9ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model09_subm/ID05WxObCFTs9.png: 512x512 4 pans, 313.9ms
Speed: 0.7ms preprocess, 313.9ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 512)

image 1/1 /Users/hugover/Desktop/CAA01/ModelBuild/model09_subm/ID06AdCmLMlkO.png: 512x512 3 boils, 311.7ms
Speed: 0.7ms preprocess, 311.7ms inference, 1.9ms postprocess per image at shape 

,ID,Target
0,ID00qprY_boil,0
1,ID00qprY_pan,1
2,ID01AciUc_boil,0
3,ID01AciUc_pan,2
4,ID0328D_boil,0
5,ID0328D_pan,4
6,ID05WxObCFTs9_boil,0
7,ID05WxObCFTs9_pan,4
8,ID06AdCmLMlkO_boil,3
9,ID06AdCmLMlkO_pan,0


In [35]:
submission.to_csv("zindi_files/model09_submission.csv", index=False)